In [289]:
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
import contractions
#import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MaxAbsScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import Perceptron

import nltk
from nltk.corpus import stopwords
from nltk import tokenize
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer 


nltk.download('punkt', quiet = True)
nltk.download('wordnet', quiet = True)
nltk.download('averaged_perceptron_tagger', quiet = True)
nltk.download('stopwords', quiet = True)
nltk.download('omw-1.4')


[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\kes\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [290]:
! pip install zstandard

In [291]:
## using in block processing to make data visualize easier
# def preprocess_data(df):
#     # lower case
#     df['Text'] = df['Text'].str.lower()

#     # remove url
#     df['Text'] = df['Text'].replace(r'[\S]+\.(net|com|org|info|edu|gov|uk|de|ca|jp|fr|au|us|ru|ch|it|nel|se|no|es|mil)[\S]*\s?', 'URL', regex=True)

#     # remove non-alphabetical
#     df['Text'] = df['Text'].str.replace('[^a-zA-Z\']', ' ', regex=True).str.strip()


#     # remove extra spaces
#     df['Text'] = df['Text'].str.replace(' +', ' ', regex=True).str.strip()

#     # #expand contractions
#     df['Text'] = df['Text'].apply(lambda x: contractions.fix(x))
#     # df['Text'] = df['Text'].apply(lambda x: [contractions.fix(word) for word in x.split()])
#     # #join back words
#     # df['Text'] = [' '.join(map(str, l)) for l in X['Text']]

#     # #remove html and url form reviews
#     # df['Text'] = df['Text'].str.replace(r'\s*https?://\S+(\s+|$)', '', regex=True).str.strip()
#     # #remove non-alphabetical characters
#     # df['Text'] = df['Text'].str.replace('[^a-zA-Z]', ' ', regex=True)
#     # #remove extra spaces
#     # df['Text'] = df['Text'].replace(r'\s+', ' ', regex=True)
#     return df

In [292]:
def tokenize_and_remove_stopwords(X):
    stop_words = set(stopwords.words('english'))
    X['tokens'] = X['Text'].apply(tokenize.word_tokenize)
    return X['tokens'].apply(lambda x: [word for word in x if word not in stop_words])

In [293]:
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

In [294]:
def vectorize_tfidf(X):
    tfidf=TfidfVectorizer(min_df = 50, max_df=0.95, ngram_range = (1,3), max_features=1500, norm='l2')
    X_data = tfidf.fit_transform(X)
    return X_data.toarray()

In [295]:
#read csv file and convert to dataframe
df = pd.read_pickle('data/2019.pkl.zst',compression = 'zstd')

In [296]:
 # remnove nan from the df
df = df.dropna(how="any")

In [297]:
df

,Title,Docket,Argued,Decided,Text
0,AARON J. SCHOCK v. UNITED STATES(2019),No. 18-406,,"February 19, 2019",United States Supreme Court AARON J. SCHOCK v...
1,"ABU-ALI ABDUR'RAHMAN, ET AL. v. TONY PARKER,...",No. 18-8332,,"May 13, 2019",United States Supreme Court ABU-ALI ABDUR'RAHM...
2,AIR &AMP; LIQUID SYSTEMS CORP. ET AL. v. DEVRI...,No. 17-1104,"October 10, 2018","March 19, 2019",United States Supreme Court AIR &AMP; LIQUID S...
3,AMERICAN LEGION ET AL. v. AMERICAN HUMANIST AS...,No. 17-1717,"February 27, 2019","June 20, 2019",United States Supreme Court AMERICAN LEGION ET...
4,APPLE INC. v. PEPPER ET AL.(2019),No. 17-204,"November 26, 2018","May 13, 2019",United States Supreme Court APPLE INC. v. PEPP...
...,...,...,...,...,...
62,TIMBS v. INDIANA(2019),No. 17-1091,"November 28, 2018","February 20, 2019",United States Supreme Court TIMBS v. INDIANA(2...
63,VIRGINIA HOUSE OF DELEGATES ET AL. v. BETHUNE-...,No. 18-281,"March 18, 2019","June 17, 2019",United States Supreme Court VIRGINIA HOUSE OF ...
64,"VIRGINIA URANIUM, INC., ET AL. v. WARREN ET AL...",No. 16-1275,"November 5, 2018","June 17, 2019","United States Supreme Court VIRGINIA URANIUM, ..."
65,"WALTER DANIEL, INDIVIDUALLY AND AS PERSONAL RE...",No. 18-460,,"May 20, 2019","United States Supreme Court WALTER DANIEL, IND..."


In [298]:
df['Text'].head()[0]

'United States Supreme Court AARON J. SCHOCK  v.  UNITED STATES(2019) No. 18-406 Argued: Decided: February 19, 2019         The petition for a writ of certiorari is denied.      Statement of Justice Sotomayor respecting the denial of certiorari.      Petitioner Aaron Schock, a former Congressman from Illinois, asks us to decide whether he may immediately appeal, as a collateral order, the denial of his motion to dismiss part of a criminal indictment against him for running afoul of the Constitution\'s Rulemaking Clause.  See Art. I, S5.  He argues that certain charges against him would require the District Court for the Central District of Illinois to interpret internal rules adopted by the House of Representatives to govern its own Members, and thus would violate separation-of-powers doctrine.  The Court of Appeals for the Seventh Circuit held that denials of such Rulemaking Clause challenges are not collateral orders subject to immediate appeal, 891 F. 3d 334 (2018), in disagreement 

In [299]:
def remove_space(match_obj):
    if match_obj.group() is not None:
        return match_obj.group().replace(' ','|')

In [300]:
# Disabled as it might be important later on
# # lower case
# df['Text'] = df['Text'].str.lower()

# remove url
df['Text'] = df['Text'].str.replace(r'[\S]+\.(net|com|org|info|edu|gov|uk|de|ca|jp|fr|au|us|ru|ch|it|nel|se|no|es|mil)[\S]*\s?', 'URL', regex=True)


# remove '|' so we can use it as a unique space seperator for cases
df['Text'] = df['Text'].str.replace(r'|', '', regex=True)


df['Text'] = df['Text'].str.replace(r'U\. S\. C\.', 'U.S.C.', regex=True)
df['Text'] = df['Text'].str.replace(r'U\. S\.', 'U.S.', regex=True)
df['Text'] = df['Text'].str.replace(r'No\. ', 'No.', regex=True)
r"F\.( \d+-?\w*,?)+( \([A-Z]*\d* ?\d*\),?)*"
# df['Text'] = df['Text'].str.replace(r"F\.( \d+-?\w*,?)+( \([A-Z]*\d* ?\d*\),?)*", remove_space, regex=True)
text = []
for element in df['Text']:
    element = re.sub(r"\bF\.( \d+-?\w*,?)+( \([A-Z]*\d* ?\d*\),?)*", remove_space, element)
    element = re.sub(r"\b(\d+ )?U\.S\.C\. [A-Z]+\d*", remove_space, element)
    element = re.sub(r"\b(\d+ )?U\.S\.(( |-)\(?\d+,?\)?)+", remove_space, element)
    element = re.sub(r"\bArt\. [A-Z]+,?( [A-Z]\d+)?", remove_space, element)
    element = re.sub(r"\b\d{4} [A-Z]+ \d+,?( \*\d*)?", remove_space, element)
    element = re.sub(r"\bn\.( \d+)+( \(.+?\))?", remove_space, element)
    element = re.sub(r"\bPp\. [0-9\-]+", remove_space, element)
    # element = re.sub(r"\b([A-Z]+[a-z]*[A-Z\.']+,? )+v\.( [A-Z]+[a-z]*[A-Z\.']+,?)+ ?", remove_space, element)
    element = re.sub(r"\b\d+ U\.S.[_,\- ]*\(\d+\)", remove_space, element)
    text.append(element)
df['Text'] = text
# Disabled as legal cases need punctuations to work
# # remove non-alphabetical
# df['Text'] = df['Text'].str.replace('[^a-zA-Z0-9\'\".!()]', ' ', regex=True).str.strip()


# remove extra spaces
df['Text'] = df['Text'].str.replace(' +', ' ', regex=True).str.strip()

# This is making U. S. as You. S.
# # #expand contractions
# df['Text'] = df['Text'].apply(lambda x: contractions.fix(x))

df['Text'].head()[0]

'United States Supreme Court AARON J. SCHOCK v. UNITED STATES(2019) No.18-406 Argued: Decided: February 19, 2019 The petition for a writ of certiorari is denied. Statement of Justice Sotomayor respecting the denial of certiorari. Petitioner Aaron Schock, a former Congressman from Illinois, asks us to decide whether he may immediately appeal, as a collateral order, the denial of his motion to dismiss part of a criminal indictment against him for running afoul of the Constitution\'s Rulemaking Clause. See Art.|I,|S5. He argues that certain charges against him would require the District Court for the Central District of Illinois to interpret internal rules adopted by the House of Representatives to govern its own Members, and thus would violate separation-of-powers doctrine. The Court of Appeals for the Seventh Circuit held that denials of such Rulemaking Clause challenges are not collateral orders subject to immediate appeal, 891 F.|3d|334|(2018), in disagreement with at least one other 

In [301]:
print(df['Text'].head()[2])

United States Supreme Court AIR &AMP; LIQUID SYSTEMS CORP. ET AL. v. DEVRIES, INDIVIDUALLY AND AS ADMINISTRATRIX OF THE ESTATE OF DEVRIES, DECEASED, ET AL.(2019) No.17-1104 Argued: October 10, 2018Decided: March 19, 2019 Petitioners produced equipment for three Navy ships. The equipment required asbestos insulation or asbestos parts to function as intended, but the manufacturers did not always incorporate the asbestos into their products. Instead, the manufacturers delivered much of the equipment to the Navy without asbestos, and the Navy later added the asbestos to the equipment. Two Navy veterans, Kenneth McAfee and John DeVries, were exposed to asbestos on the ships and developed cancer. They and their wives sued the manufacturers, alleging that the asbestos exposure caused the cancer and contending that the manufacturers were negligent in failing to warn about the dangers of asbestos in the integrated products. Raising the "bare-metal defense," the manufacturers argued that they sh

In [302]:
# we go through the text and concatinating legal case nombers and special names
df['tokens'] = df['Text'].str.split() 
df['tokens'][0]

['United',
 'States',
 'Supreme',
 'Court',
 'AARON',
 'J.',
 'SCHOCK',
 'v.',
 'UNITED',
 'STATES(2019)',
 'No.18-406',
 'Argued:',
 'Decided:',
 'February',
 '19,',
 '2019',
 'The',
 'petition',
 'for',
 'a',
 'writ',
 'of',
 'certiorari',
 'is',
 'denied.',
 'Statement',
 'of',
 'Justice',
 'Sotomayor',
 'respecting',
 'the',
 'denial',
 'of',
 'certiorari.',
 'Petitioner',
 'Aaron',
 'Schock,',
 'a',
 'former',
 'Congressman',
 'from',
 'Illinois,',
 'asks',
 'us',
 'to',
 'decide',
 'whether',
 'he',
 'may',
 'immediately',
 'appeal,',
 'as',
 'a',
 'collateral',
 'order,',
 'the',
 'denial',
 'of',
 'his',
 'motion',
 'to',
 'dismiss',
 'part',
 'of',
 'a',
 'criminal',
 'indictment',
 'against',
 'him',
 'for',
 'running',
 'afoul',
 'of',
 'the',
 "Constitution's",
 'Rulemaking',
 'Clause.',
 'See',
 'Art.|I,|S5.',
 'He',
 'argues',
 'that',
 'certain',
 'charges',
 'against',
 'him',
 'would',
 'require',
 'the',
 'District',
 'Court',
 'for',
 'the',
 'Central',
 'District',


In [303]:
df['tokens'][1]

['United',
 'States',
 'Supreme',
 'Court',
 'ABU-ALI',
 "ABDUR'RAHMAN,",
 'ET',
 'AL.',
 'v.',
 'TONY',
 'PARKER,',
 'COMMISSIONER,',
 'TENNESSEE',
 'DEPARTMENT',
 'OF',
 'CORRECTIONS,',
 'ET',
 'AL.(2019)',
 'No.18-8332',
 'Argued:',
 'Decided:',
 'May',
 '13,',
 '2019',
 'The',
 'petition',
 'for',
 'a',
 'writ',
 'of',
 'certiorari',
 'is',
 'denied.',
 'Justice',
 'Sotomayor,',
 'dissenting',
 'from',
 'denial',
 'of',
 'certiorari.',
 'I',
 'have',
 'already',
 'explained',
 'my',
 'opposition',
 'to',
 'the',
 '"perverse',
 'requirement',
 'that',
 'inmates',
 'offer',
 'alternative',
 'methods',
 'for',
 'their',
 'own',
 'executions."',
 'McGehee',
 'v.',
 'Hutchinson,',
 '581|U.S.|___,|___|(2017)',
 '(opinion',
 'dissenting',
 'from',
 'denial',
 'of',
 'application',
 'for',
 'stay',
 'and',
 'denial',
 'of',
 'certiorari)',
 '(slip',
 'op.,',
 'at',
 '2);',
 'see',
 'generally',
 'Glossip',
 'v.',
 'Gross,',
 '576|U.S.|___,|___-___|(2015)',
 '(slip',
 'op.,',
 'at',
 '13-15

In [306]:
new_tokens = []
for case in df['tokens']:
    new_token = []
    case_found = False
    for tokenIdx in range(len(case)):
        new_token.append(case[tokenIdx])
        # Handling cases
        #United States v. Rostenkowski, 59 F. 3d 1291, 1297 (CADC 1995).
        #United States Supreme Court AARON J. SCHOCK v. UNITED STATES(2019) No. 18-406
        if case[tokenIdx] == 'v.':
            case_found = True
        elif case_found and (case[tokenIdx].startswith('No.') or case[tokenIdx][0].islower() or case[tokenIdx][0].isnumeric() or case[tokenIdx-1].lower().startswith('al.')):
            # we need to deal with this
            last_word = new_token.pop()
            castStr = ''
            while len(new_token) > 0 and (new_token[-1] == 'v.' or (new_token[-1].lower() != 'see' and new_token[-1][0].isupper())) :
                castStr =  new_token.pop() + '|' + castStr
            new_token.append(castStr[:-1])
            case_found = False
            new_token.append(last_word)
    new_tokens.append(new_token)
new_tokens

[['United|States|Supreme|Court|AARON|J.|SCHOCK|v.|UNITED|STATES(2019)',
  'No.18-406',
  'Argued:',
  'Decided:',
  'February',
  '19,',
  '2019',
  'The',
  'petition',
  'for',
  'a',
  'writ',
  'of',
  'certiorari',
  'is',
  'denied.',
  'Statement',
  'of',
  'Justice',
  'Sotomayor',
  'respecting',
  'the',
  'denial',
  'of',
  'certiorari.',
  'Petitioner',
  'Aaron',
  'Schock,',
  'a',
  'former',
  'Congressman',
  'from',
  'Illinois,',
  'asks',
  'us',
  'to',
  'decide',
  'whether',
  'he',
  'may',
  'immediately',
  'appeal,',
  'as',
  'a',
  'collateral',
  'order,',
  'the',
  'denial',
  'of',
  'his',
  'motion',
  'to',
  'dismiss',
  'part',
  'of',
  'a',
  'criminal',
  'indictment',
  'against',
  'him',
  'for',
  'running',
  'afoul',
  'of',
  'the',
  "Constitution's",
  'Rulemaking',
  'Clause.',
  'See',
  'Art.|I,|S5.',
  'He',
  'argues',
  'that',
  'certain',
  'charges',
  'against',
  'him',
  'would',
  'require',
  'the',
  'District',
  'Cou

In [202]:
df['tokens'] = df['tokens'].apply(nltk.tag.pos_tag)
df['tokens'][0]


[('United', 'NNP'),
 ('States', 'NNPS'),
 ('Supreme', 'NNP'),
 ('Court', 'NNP'),
 ('AARON', 'NNP'),
 ('J.', 'NNP'),
 ('SCHOCK', 'NNP'),
 ('v.', 'NN'),
 ('UNITED', 'NNP'),
 ('STATES(2019)', 'NNP'),
 ('No.18-406', 'NNP'),
 ('Argued:', 'NNP'),
 ('Decided:', 'NNP'),
 ('February', 'NNP'),
 ('19,', 'CD'),
 ('2019', 'CD'),
 ('The', 'DT'),
 ('petition', 'NN'),
 ('for', 'IN'),
 ('a', 'DT'),
 ('writ', 'NN'),
 ('of', 'IN'),
 ('certiorari', 'NN'),
 ('is', 'VBZ'),
 ('denied.', 'JJ'),
 ('Statement', 'NNP'),
 ('of', 'IN'),
 ('Justice', 'NNP'),
 ('Sotomayor', 'NNP'),
 ('respecting', 'VBG'),
 ('the', 'DT'),
 ('denial', 'NN'),
 ('of', 'IN'),
 ('certiorari.', 'NN'),
 ('Petitioner', 'NNP'),
 ('Aaron', 'NNP'),
 ('Schock,', 'NNP'),
 ('a', 'DT'),
 ('former', 'JJ'),
 ('Congressman', 'NN'),
 ('from', 'IN'),
 ('Illinois,', 'NNP'),
 ('asks', 'VBZ'),
 ('us', 'PRP'),
 ('to', 'TO'),
 ('decide', 'VB'),
 ('whether', 'IN'),
 ('he', 'PRP'),
 ('may', 'MD'),
 ('immediately', 'RB'),
 ('appeal,', 'VB'),
 ('as', 'IN'),
 ('a

# Establish base line performance

In [133]:
# # preprocess df['tokens'] using preocess_text function
# data = df[:100]


In [134]:
# df['tokens'] = preprocess_data(data)


In [135]:
# df['tokens'] = tokenize_and_remove_stopwords(df['tokens'])

In [136]:
stop_words = set(stopwords.words('english'))
df['tokens'] = df['Text'].apply(tokenize.word_tokenize)
df['tokens'] = df['tokens'].apply(lambda x: [word for word in x if word not in stop_words])
df['tokens'][0]

['united',
 'states',
 'supreme',
 'court',
 'aaron',
 'j.',
 'schock',
 'v.',
 'united',
 'states',
 '(',
 '2019',
 ')',
 '.',
 '18-406',
 'argued',
 ':',
 'decided',
 ':',
 'february',
 '19',
 ',',
 '2019',
 'petition',
 'writ',
 'certiorari',
 'denied',
 '.',
 'statement',
 'justice',
 'sotomayor',
 'respecting',
 'denial',
 'certiorari',
 '.',
 'petitioner',
 'aaron',
 'schock',
 ',',
 'former',
 'congressman',
 'illinois',
 ',',
 'asks',
 'us',
 'decide',
 'whether',
 'may',
 'immediately',
 'appeal',
 ',',
 'collateral',
 'order',
 ',',
 'denial',
 'motion',
 'dismiss',
 'part',
 'criminal',
 'indictment',
 'running',
 'afoul',
 'constitution',
 "'s",
 'rulemaking',
 'clause',
 '.',
 'see',
 'art',
 '.',
 ',',
 's5',
 '.',
 'argues',
 'certain',
 'charges',
 'would',
 'require',
 'district',
 'court',
 'central',
 'district',
 'illinois',
 'interpret',
 'internal',
 'rules',
 'adopted',
 'house',
 'representatives',
 'govern',
 'members',
 ',',
 'thus',
 'would',
 'violate',
 'se

In [138]:
df['tokens'] = df['tokens'].apply(lambda x: [(word, get_wordnet_pos(pos_tag)) for (word, pos_tag) in x])
df['tokens'][0]

[('united', 'a'),
 ('states', 'n'),
 ('supreme', 'a'),
 ('court', 'n'),
 ('aaron', 'n'),
 ('j.', 'n'),
 ('schock', 'n'),
 ('v.', 'n'),
 ('united', 'a'),
 ('states', 'n'),
 ('(', None),
 ('2019', None),
 (')', None),
 ('.', None),
 ('18-406', 'a'),
 ('argued', 'v'),
 (':', None),
 ('decided', 'v'),
 (':', None),
 ('february', 'a'),
 ('19', None),
 (',', None),
 ('2019', None),
 ('petition', 'n'),
 ('writ', 'n'),
 ('certiorari', 'n'),
 ('denied', 'v'),
 ('.', None),
 ('statement', 'n'),
 ('justice', 'n'),
 ('sotomayor', 'n'),
 ('respecting', 'v'),
 ('denial', 'a'),
 ('certiorari', 'n'),
 ('.', None),
 ('petitioner', 'n'),
 ('aaron', 'n'),
 ('schock', 'n'),
 (',', None),
 ('former', 'a'),
 ('congressman', 'n'),
 ('illinois', 'n'),
 (',', None),
 ('asks', 'v'),
 ('us', None),
 ('decide', None),
 ('whether', None),
 ('may', None),
 ('immediately', 'r'),
 ('appeal', 'v'),
 (',', None),
 ('collateral', 'a'),
 ('order', 'n'),
 (',', None),
 ('denial', 'a'),
 ('motion', 'n'),
 ('dismiss', 'v'),

In [139]:
lemma = WordNetLemmatizer()
df['tokens'] = df['tokens'].apply(lambda x: [lemma.lemmatize(word, tag) if tag else lemma.lemmatize(word) for word, tag in x])
df['tokens'][0]

['united',
 'state',
 'supreme',
 'court',
 'aaron',
 'j.',
 'schock',
 'v.',
 'united',
 'state',
 '(',
 '2019',
 ')',
 '.',
 '18-406',
 'argue',
 ':',
 'decide',
 ':',
 'february',
 '19',
 ',',
 '2019',
 'petition',
 'writ',
 'certiorari',
 'deny',
 '.',
 'statement',
 'justice',
 'sotomayor',
 'respect',
 'denial',
 'certiorari',
 '.',
 'petitioner',
 'aaron',
 'schock',
 ',',
 'former',
 'congressman',
 'illinois',
 ',',
 'ask',
 'u',
 'decide',
 'whether',
 'may',
 'immediately',
 'appeal',
 ',',
 'collateral',
 'order',
 ',',
 'denial',
 'motion',
 'dismiss',
 'part',
 'criminal',
 'indictment',
 'run',
 'afoul',
 'constitution',
 "'s",
 'rulemaking',
 'clause',
 '.',
 'see',
 'art',
 '.',
 ',',
 's5',
 '.',
 'argue',
 'certain',
 'charge',
 'would',
 'require',
 'district',
 'court',
 'central',
 'district',
 'illinois',
 'interpret',
 'internal',
 'rule',
 'adopt',
 'house',
 'representative',
 'govern',
 'member',
 ',',
 'thus',
 'would',
 'violate',
 'separation-of-powers',
 

In [23]:
df['tokens'] = [' '.join(map(str, l)) for l in df['tokens']]

In [24]:
#vectorize the text using tfidf vectorizer
# X_data = vectorize_tfidf(df['tokens'])
tfidf=TfidfVectorizer(min_df = 50, max_df=0.95, ngram_range = (1,3), max_features=1500, norm='l2')
X_data = tfidf.fit_transform(df['tokens'])
df['tokens'] = X_data.toarray()

In [25]:
X_data.shape

(100, 434)

In [72]:
! pip install gensim

     --------------------------------------- 23.9/23.9 MB 36.4 MB/s eta 0:00:00
  Using cached smart_open-6.2.0-py3-none-any.whl (58 kB)
  Using cached Cython-0.29.28-py2.py3-none-any.whl (983 kB)


In [26]:
import gensim.downloader as api
from gensim.models import Word2Vec

In [27]:
w2v_model = Word2Vec(df['tokens'], vector_size=300, window=5, min_count=10)

In [28]:
def create_word2vec_embeddings(X):
    embeddings = []
    for sentence in X:
        sentence_embedding = np.zeros(300)
        for word in sentence:
            if word in w2v_model.wv:
                sentence_embedding += w2v_model.wv[word]
        embeddings.append(sentence_embedding)
    return np.array(embeddings)

In [29]:
w2v_vectors = create_word2vec_embeddings(df['tokens'])

In [30]:
w2v_vectors.shape

(100, 300)

## TF-IDF Model

In [31]:
import gensim
import gensim.downloader as api

def tagged_document(list_of_list_of_words):
   for i, list_of_words in enumerate(list_of_list_of_words):
      yield gensim.models.doc2vec.TaggedDocument(list_of_words, [i])

def doc2vec_vectors(X):
    documents = list(tagged_document(X[0].split()))
    model = gensim.models.doc2vec.Doc2Vec(documents, vector_size=300, window=5, min_count=10)
    return np.array([model.infer_vector(doc.split()) for doc in X])

In [32]:
doc2vec = doc2vec_vectors(df['tokens'])

In [33]:
#find most similar cases using cosine similarity and tfidf vectors
def find_similar_cases(X, case_id, n):
    pairwise_similarities=np.dot(X,X.T)
    most_similar = pairwise_similarities[case_id].argsort()[:-n-1:-1]
    return most_similar

#find most similar cases using cosine similarity and word2vec vectors

find_similar_cases(X_data, 0, 4)


array([0, 5, 4, 8], dtype=int64)

## Word2Vec Model

In [34]:
import tensorflow as tf

from tensorflow.keras.preprocessing.sequence import pad_sequences

In [40]:
#find the max sequence length of word2vec vectors\
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer=Tokenizer()
tokenizer.fit_on_texts(df['tokens'])
tokenized_documents=tokenizer.texts_to_sequences(df['tokens'])
tokenized_paded_documents=pad_sequences(tokenized_documents,maxlen=5000,padding='post')
vocab_size=len(tokenizer.word_index)+1


(5000,)


In [ ]:
w2v_vectors = create_word2vec_embeddings(df['tokens'])

## BERT Model

In [41]:
! pip install sentence-transformers
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('bert-base-nli-mean-tokens')
sentence_embeddings = model.encode(df['tokens'])

     ---------------------------------------- 86.0/86.0 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 5.5/5.5 MB 8.8 MB/s eta 0:00:00
     -------------------------------------- 167.2/167.2 MB 6.5 MB/s eta 0:00:00
     ---------------------------------------- 1.1/1.1 MB 11.6 MB/s eta 0:00:00
     ---------------------------------------- 1.1/1.1 MB 10.2 MB/s eta 0:00:00
  Using cached huggingface_hub-0.10.1-py3-none-any.whl (163 kB)
     -------------------------------------- 151.6/151.6 kB 8.8 MB/s eta 0:00:00
  Using cached tokenizers-0.13.1-cp39-cp39-win_amd64.whl (3.3 MB)
     ---------------------------------------- 2.5/2.5 MB 9.9 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125930 sha256=695baba5210bbbaf5df1b9d4bb5ab3754c05bb66f78afacf770b8cb2529f2050
  Stored in directory: c:\users\a

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.95k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/399 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [42]:
from sklearn.metrics.pairwise import cosine_similarity

similar = cosine_similarity(
    [sentence_embeddings[0]],
    sentence_embeddings[:]
)
ind = np.argsort(similar[0])[::-1][:5]

In [43]:
ind

array([ 0, 98,  2,  7, 88], dtype=int64)